> **Importing Core Libraries for Model Evaluation**

This cell imports the essential libraries and modules needed for the experiments.

**What to check:**

- `math` for mathematical operations.
- `torch` for deep learning and tensor operations.
- `AutoTokenizer` and `AutoModelForCausalLM` from `transformers` for model and tokenizer loading.

In [ ]:
import math
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

> **Model Selection: Choosing the LLM for Experiments**

This cell specifies the model name to be used throughout the notebook. The chosen model should be small enough to run on limited hardware but representative for scaling experiments.

**What to check:**

- Assign a string with the Hugging Face model name (e.g., `'TinyLlama/TinyLlama-1.1B-Chat-v1.0'`) to a variable like `MODEL_NAME`.

In [ ]:
# ─── SETTINGS ────────────────────────────────────────────────────────────────
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

> **Model and Tokenizer Loader: Device and Precision Handling**

This function loads the model and tokenizer, moves the model to the correct device (GPU if available, otherwise CPU), and sets the appropriate numerical precision. The model is set to evaluation mode.

**What to implement:**

- Detect if CUDA (GPU) is available and set the device accordingly.
- Choose `torch.float16` if using CUDA, otherwise use `torch.float32`.
- Load the tokenizer and model from the specified model name.
- Move the model to the selected device and set it to evaluation mode.
- Return the tokenizer, model, and device.

In [ ]:
def load_model_and_tokenizer(model_name: str):
    """
    TODO:
      - Load tokenizer & model from `model_name`
      - Move model to GPU if available, choose float16 for CUDA else float32
      - Set model to eval mode
      - Return (tokenizer, model, device)
    """
    raise NotImplementedError


> **Text Selection: Providing a Sample Passage**

This function returns a sample passage of about 100 tokens, ideally from Wikipedia or a similar source. This passage will be used as the input prompt for all experiments.

**What to implement:**

- Choose or paste a paragraph of about 100 tokens.
- Return the text as a string.

In [ ]:
def select_text():
    """
    TODO:
      - Pick or paste a ~100-token passage (e.g. a Wiki snippet)
      - Return (text_str)
    """
    raise NotImplementedError

> **Tokenization with Labels: Preparing Model Inputs**

This function tokenizes the selected text and prepares the input tensors for the model, including setting up the labels for language modeling.

**What to implement:**

- Tokenize the text using the provided tokenizer with `return_tensors='pt'`.
- Set `labels = input_ids` in the returned dictionary.
- Return the inputs dictionary and the input length (number of tokens).

In [ ]:
def tokenize_with_labels(tokenizer, text: str):
    """
    TODO:
      - Tokenize `text` with return_tensors="pt"
      - Prepare inputs and set labels = input_ids
      - Return (inputs_dict, input_len)
    """
    raise NotImplementedError

> **Peak Memory and Loss Calculation: Measuring Resource Usage**

This function runs a forward pass of the model on the provided inputs and measures the peak GPU memory usage (if CUDA is available) and the loss value.

**What to implement:**

- If using CUDA, reset peak memory stats before running the model.
- Run the model in `torch.no_grad()` mode to avoid gradient computation.
- After the forward pass, if using CUDA, synchronize and get the peak memory allocated (in MiB).
- Return the peak memory (MiB) and the loss value.

In [ ]:
def compute_peak_memory_and_loss(model, inputs, device):
    """
    TODO:
      - Reset peak memory stats if CUDA
      - Run model(**inputs) under torch.no_grad()
      - Sync CUDA if needed
      - Retrieve peak memory via torch.cuda.max_memory_allocated (in MiB)
      - Return (peak_mem_mib, loss_value)
    """
    raise NotImplementedError

> **Perplexity Computation: Interpreting Model Loss**

This function converts the model's loss value into perplexity, a standard metric for evaluating language models.

**What to implement:**

- Use `math.exp(loss)` to compute perplexity from the loss value.
- Return the perplexity.

In [ ]:
def compute_perplexity(loss: float):
    """
    TODO:
      - Compute and return math.exp(loss)
    """
    raise NotImplementedError

> **Forward Pass Timing: Measuring Inference Latency**

This function measures the average time taken for the model to perform a forward pass (inference) on the input data. It includes warmup runs to stabilize performance and reports the mean latency over several timed runs.

**What to implement:**

- Move all input tensors to the correct device.
- Run a few warmup passes (not timed) to stabilize performance.
- For each timed run:

  - Record the start time.
  - Run the model in `torch.no_grad()` mode.
  - If using CUDA, synchronize after each run.
  - Record the end time and compute the latency.
- Return the average latency in seconds.

In [ ]:
def time_forward(model, inputs: Dict[str, torch.Tensor], device, num_warmup: int = 1, num_runs: int = 3):
    """
    TODO:
      1) Move all tensors in `inputs` to `device`.
      2) Warmup (no timing): under torch.no_grad(), run a small number of forward passes
         (num_warmup times) to stabilize kernels/clocks:
             _ = model(**inputs_on_device)
         If CUDA, call torch.cuda.synchronize() after warmup.
      3) Timed runs: create an empty list `latencies = []`.
         For _ in range(num_runs):
           - Record start time with time.perf_counter()
           - Under torch.no_grad(), run a forward pass:
                 _ = model(**inputs_on_device)
           - If CUDA, call torch.cuda.synchronize()
           - Record end time with time.perf_counter()
           - Append (end - start) to `latencies`.
      4) Return the average latency in seconds:
             return sum(latencies) / max(len(latencies), 1)
    """
    raise NotImplementedError

> **Context Length Sweep: Scaling Analysis with Input Size**

This function systematically increases the input (context) length and measures how peak memory usage and inference latency scale. It builds longer input texts, runs the model, and visualizes the results.

**What to implement:**

- Determine the model's maximum context length from `model.config` (default to 2048 if unavailable).

- Filter the target context lengths to those that fit within the model's window.
- Build input texts of increasing length by repeating the base passage.
- For each target length:

  - Tokenize and prepare inputs.
  - Move tensors to the correct device.
  - Measure peak memory and latency using the functions above.
  - Print a summary for each run.
- Plot memory and latency vs. input length using matplotlib.
- Return a dictionary with the results.

In [ ]:
import matplotlib.pyplot as plt

def sweep_context_length(
    tokenizer,
    model,
    device,
    targets=(64, 256, 1024),
    base_text=None,
    build_text_fn=None,
    warmup=1,
    runs=3,
    plot=True,
):
    """
    TODO:
      1) Determine the model's maximum context length:
         - Read `max_position_embeddings` from `model.config` (fallback to 2048).
         - Filter `targets` to those <= `max_len - 2` (leave room for special tokens).
         - If no valid targets remain, raise a ValueError.

      2) Choose a base paragraph:
         - If `base_text` is None, call `select_text()`; if unavailable, raise with a helpful message.

      3) Build texts of desired token lengths:
         - Implement a builder that repeats `base_text` until the tokenized length >= `target_tokens`.
         - Allow overriding via `build_text_fn(tokenizer, base_text, target_tokens)`.

      4) For each target length:
         - Build the text and tokenize with `tokenize_with_labels(tokenizer, text)` to get `(inputs, input_len)`.
         - Move all tensors in `inputs` to `device`.
         - Compute peak memory (MiB) and loss using `compute_peak_memory_and_loss(model, inputs, device)`.
         - Measure forward (prefill) latency using `time_forward(model, inputs, device, num_warmup=warmup, num_runs=runs)`.
         - Append `input_len`, `peak_mib`, and `latency_s` to running lists.
         - Print a concise summary line for each target.

      5) If `plot` is True, create two matplotlib plots (one per figure):
         - Peak memory (MiB) vs input tokens.
         - Forward latency (s) vs input tokens.
         - Use simple line plots with markers and grid; do not specify custom colors.

      6) Return a results dictionary with:
         - 'lengths': list of actual input token counts
         - 'peak_mib': list of peak memory values (MiB)
         - 'latency_s': list of forward latencies (s)
         - 'max_ctx': the model’s max context length
         - 'targets_used': the filtered list of targets
    """
    raise NotImplementedError

> **Precision Comparison: Memory and Perplexity Across dtypes**

This function compares the model's memory usage and perplexity when run with different numerical precisions (float16 and float32). It reloads the model for each precision, runs evaluation, and summarizes the trade-offs.

**What to implement:**

- Prepare a short evaluation paragraph (use the text selection function).

- Tokenize the text once for all precisions.
- For each precision:

  - Reload the model and tokenizer with the correct dtype.
  - Move inputs to the correct device.
  - Measure peak memory and loss, then compute perplexity.
  - Store results in a dictionary.
- Return the results dictionary.

In [ ]:
from typing import Dict, Iterable, Callable, Tuple

def compare_precision_memory_perplexity(
    model_name: str,
    tokenizer,
    load_with_dtype_fn: Callable[[str], Tuple],  # expected to return (tokenizer, model, device)
    precisions: Iterable[str] = ("float16", "float32"),
    base_text: str = None,
) -> Dict[str, Dict[str, float]]:
    """
    TODO:
      1) Prepare a short evaluation paragraph:
         - If `base_text` is None, call `select_text()` to get a default paragraph.

      2) Tokenize once with the provided `tokenizer`:
         - Use `tokenize_with_labels(tokenizer, text)` to get `(inputs_s, Ls)`.

      3) For each precision in `precisions`:
         - Reload the model with that dtype via: tok_p, mod_p, dev_p = load_with_dtype_fn(precision)
           (Assume this function *honors* dtype; on CPU it may force float32.)
         - Move pre-tokenized inputs to the appropriate device:
               inp = {k: v.to(dev_p) for k, v in inputs_s.items()}
         - Measure peak memory (MiB) and loss:
               peak_mib, loss_val = compute_peak_memory_and_loss(mod_p, inp, dev_p)
         - Convert loss to perplexity:
               ppl = compute_perplexity(loss_val)
         - Save into a results dict keyed by precision:
               results[precision] = {"peak_mib": peak_mib, "ppl": ppl}

      4) Return the results dict.

    Returns:
      Dict[str, Dict[str, float]] like:
        {"float16": {"peak_mib": 123.4, "ppl": 15.2}, "float32": {...}}
    """
    raise NotImplementedError

> **Batch Size Sweep: Analyzing Memory Scaling with Batch Size**

This function evaluates how peak memory usage changes as the batch size increases. It creates batches of repeated input text, runs the model, and plots the relationship between batch size and memory consumption.

**What to implement:**

- Prepare a base paragraph for batching.
- For each batch size:

  - Create a batch of repeated texts and tokenize with padding and truncation.
  - Move tensors to the correct device.
  - Measure peak memory using the earlier function.
  - Print a summary for each batch size.
- Plot memory vs. batch size using matplotlib.
- Return a dictionary with the results.

In [ ]:
def sweep_batch_memory(
    tokenizer,
    model,
    device,
    batch_sizes=(1, 2, 4),
    base_text=None,
    target_len=128,
    plot=True,
):
    """
    TODO:
      1) Choose a base paragraph:
         - If `base_text` is None, call `select_text()` to get a default paragraph.

      2) Implement a helper to build batched inputs:
         - Given (tokenizer, text, bs, max_len), tokenize a list of the same `text`
           repeated `bs` times with:
               return_tensors='pt', padding='longest', truncation=True, max_length=target_len
         - Set enc['labels'] = enc['input_ids'].clone()

      3) For each batch size in `batch_sizes`:
         - Build the batch with the helper.
         - Move all tensors to `device`.
         - Measure peak memory (MiB) using `compute_peak_memory_and_loss(model, enc, device)`.
         - Record (bs, peak_mib) and print a concise summary line.

      4) If `plot` is True, produce a single matplotlib figure of:
         - x = batch size
         - y = peak MiB
         - Use a simple line plot with markers and grid; do not set custom colors.

      5) Return a results dictionary:
         {
           "batch_sizes": [...],
           "peak_mib": [...],
           "target_len": target_len,
         }
    """
    raise NotImplementedError

> **Experiment Runner: Orchestrating All Analyses**

This function coordinates the entire experiment workflow. It loads the model and tokenizer, prepares the input, runs baseline measurements, and then executes the context length sweep, precision comparison, and batch size sweep. The results are returned in a structured dictionary for further analysis or reporting.

**What to implement:**

- Load the model and tokenizer using your loader function.
- Select and tokenize a baseline paragraph.
- Run baseline measurements: peak memory, loss, perplexity, and forward latency.
- Run the context length sweep, precision comparison, and batch size sweep functions.
- Return a dictionary with all results.

In [ ]:
def start():
    if "load_with_dtype" not in globals():
        def load_with_dtype(dtype_str: str):
            tok_p, mod_p, dev_p = load_model_and_tokenizer(MODEL_NAME)
            return tok_p, mod_p, dev_p


    # --- 1) Load model/tokenizer ---
    tokenizer, model, device = load_model_and_tokenizer(MODEL_NAME)
    print(f"[load] device={device}")

    # --- 2) Select & tokenize a baseline paragraph ---
    text = select_text()
    inputs, input_len = tokenize_with_labels(tokenizer, text)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    print(f"[tokenize] tokens={input_len}")

    # --- 3) Baseline: peak memory & loss ---
    peak_mib, loss = compute_peak_memory_and_loss(model, inputs, device)
    ppl = compute_perplexity(loss)
    fwd_lat = time_forward(model, inputs, device)
    print(f"[baseline] peak={peak_mib:.1f} MiB | loss={loss:.3f} | ppl={ppl:.3f} | fwd={fwd_lat:.3f}s")

    # --- A) Context-length sweep ---
    ctx_results = sweep_context_length(
        tokenizer=tokenizer,
        model=model,
        device=device,
    )

    # --- B) Precision comparison ---
    prec_results = compare_precision_memory_perplexity(
        model_name=MODEL_NAME,
        tokenizer=tokenizer,
        load_with_dtype_fn=load_with_dtype,
        precisions=("float16", "float32"),
        base_text=text,
    )

    # --- C) Batch-size sweep ---
    batch_results = sweep_batch_memory(
        tokenizer=tokenizer,
        model=model,
        device=device,
    )

    return {
        "baseline": {
            "tokens": int(input_len),
            "peak_mib": float(peak_mib),
            "loss": float(loss),
            "ppl": float(ppl),
            "forward_latency_s": float(fwd_lat),
        },
        "context_sweep": ctx_results,
        "precision_compare": prec_results,
        "batch_sweep": batch_results,
    }


In [ ]:
start()